In [16]:
import talib
from talib.abstract import Function
import numpy as np
import pandas as pd

from gym import Space
from copy import copy
from abc import abstractmethod
from typing import Union, List, Callable, Dict

from tensortrade.features import FeatureTransformer


class TAlibIndicator(FeatureTransformer):
    """Adds one or more TAlib indicators to a data frame, based on existing open, high, low, and close column values."""

    def __init__(self, indicators: List[str], lows: Union[List[float], List[int]] = None, highs: Union[List[float], List[int]] = None, **kwargs):
        indicators = self._error_check(indicators)
        self._indicator_names = [indicator.upper() for indicator in indicators]
        self._indicators = [getattr(talib, name.split('-')[0]) for name in self._indicator_names]
        # Here we get the stats for each indicator for TA-Lib
        self._stats = {indicator:self._get_stats(indicator) for indicator in self._indicator_names}
        
    def _error_check(self, a):
        err_indexes = []
        for n, i in enumerate(a):
            if i == "BBAND":
                a[n] = "BBANDS"
            elif i == "BB":
                pass
            elif i == "RIS":
                a[n] = "RSI"
            elif i == "":
                err_indexes.append(n)
            elif i == None:
                err_indexes.append(n)
        for n in sorted(err_indexes, reverse=True):
            del a[n]
        return a
    
    def _get_stats(self, indicator_name:str) -> Dict:
        """ Get the relavent indicator information.

        Parameters:
        -------
        code: code of symbol (required)
            get help information of a symbol
        """
        if indicator_name is None:
            print("Usage: help_indicator(symbol), symbol is indicator name")
            return {
                "parameters": {},
                "inputs": []
            }
        else:
            upper_code = indicator_name.upper()
            if upper_code not in talib.get_functions():
                print(f"ERROR: indicator {upper_code} not in list")
                return {
                    "parameters": {},
                    "inputs": []
                }
            else:
                func = Function(upper_code)
                parameters = dict(func.parameters)
                inputs = list(func.input_names.values())
                return {
                    "parameters": parameters,
                    "inputs": inputs
                }

    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        for idx, indicator in enumerate(self._indicators):
            indicator_name = self._indicator_names[idx]
            indicator_params = self._stats[indicator_name]['parameters']
            indicator_args = [X[arg].values for arg in self._stats[indicator_name]["inputs"]]
            
            if indicator_name == 'BBANDS':
                upper, middle, lower = indicator(*indicator_args,**indicator_params)

                X["bb_upper"] = upper
                X["bb_middle"] = middle
                X["bb_lower"] = lower
            else:
                try:
                    value = indicator(*indicator_args,**indicator_params)

                    if type(value) == tuple:
                        X[indicator_name] = value[0][0]
                    else:
                        X[indicator_name] = value

                except:
                    X[indicator_name] = indicator(*indicator_args,**indicator_params)[0]

        return X.dropna()


In [17]:
import pandas as pd

from tensortrade.features.scalers import MinMaxNormalizer, ComparisonNormalizer, PercentChangeNormalizer
from tensortrade.features.stationarity import FractionalDifference

ohlcv_data = pd.read_csv('./data/Coinbase_BTCUSD_1h.csv', skiprows=1)
ohlcv_data = ohlcv_data[['open','high','low','close','volume']]

In [18]:
taindicator = TAlibIndicator(indicators=["BBAND", "RSI", "EMA", "SMA", "", None])

In [19]:
taindicator.transform(ohlcv_data)

,open,high,low,close,volume,bb_upper,bb_middle,bb_lower,RSI,EMA,SMA
29,8166.38,8171.00,8160.00,8165.23,774938.22,8182.976334,8153.200,8123.423666,66.977004,8022.096000,8022.096000
30,8156.00,8166.38,8143.60,8166.38,1286322.61,8182.601904,8159.902,8137.202096,67.108686,8031.404645,8026.446000
31,8171.27,8171.50,8154.40,8156.00,941053.53,8173.935749,8163.408,8152.880251,64.604552,8039.443055,8029.946000
32,8167.11,8171.59,8166.98,8171.27,2099430.35,8176.809286,8165.886,8154.962714,66.580198,8047.948019,8036.458667
33,8162.44,8168.00,8150.11,8167.11,1398796.61,8175.258866,8165.198,8155.137134,65.507465,8055.635889,8043.208333
34,8136.93,8167.50,8132.50,8162.44,2620938.73,8174.938544,8164.640,8154.341456,64.255860,8062.526477,8049.155667
35,8149.26,8155.00,8133.58,8136.93,1518457.68,8182.815427,8158.750,8134.684573,57.763423,8067.326704,8055.663000
36,8139.89,8150.00,8115.62,8149.26,3126810.06,8182.658177,8157.402,8132.145823,59.873801,8072.612723,8061.632000
37,8171.58,8171.58,8139.01,8139.89,3154582.99,8175.038355,8151.126,8127.213645,57.521656,8076.953193,8066.741000
38,8172.84,8173.00,8160.15,8171.58,2741876.33,8178.456832,8152.020,8125.583168,62.838864,8083.058148,8073.793667


In [20]:


from math import log
import pandas as pd
import numpy as np

from gym import Space
from copy import copy
from typing import Union, List, Tuple
from loguru import logger
from tensortrade.features.feature_transformer import FeatureTransformer


class StandardNormalizer(FeatureTransformer):
    """A transformer for normalizing values within a feature pipeline by removing the mean and scaling to unit variance."""

    def __init__(self, columns: Union[List[str], str, None] = None, feature_min=0, feature_max=1, inplace=True):
        """
        Arguments:
            columns (optional): A list of column names to normalize.
            feature_min (optional): The minimum value in the range to scale to.
            feature_max (optional): The maximum value in the range to scale to.
            inplace (optional): If `False`, a new column will be added to the output for each input column.
        """
        super().__init__(columns=columns, inplace=inplace)

        self._feature_min = feature_min
        self._feature_max = feature_max

        if feature_min >= feature_max:
            raise ValueError("feature_min must be less than feature_max")

        self._history = {}

    def reset(self):
        self._history = {}

    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        if self.columns is None:
            self.columns = list(X.select_dtypes('number').columns)
        
        for column in self.columns:
            if self._inplace == True:
                X[column] = (X[column] - X[column].mean())/X[column].std()
            else:
                X[f"{column}_scaled"] = (X[column] - X[column].mean())/X[column].std()
            
        return X.dropna()


In [21]:
standard = StandardNormalizer()
transformed = standard.transform(ohlcv_data)

In [23]:
transformed

,open,high,low,close,volume,bb_upper,bb_middle,bb_lower,RSI,EMA,SMA
0,0.278072,0.263818,0.286887,0.273180,-0.576874,NaN,NaN,NaN,NaN,NaN,NaN
1,0.254132,0.267973,0.272262,0.277999,-0.231663,NaN,NaN,NaN,NaN,NaN,NaN
2,0.250540,0.242045,0.268624,0.254058,-0.510299,NaN,NaN,NaN,NaN,NaN,NaN
3,0.256723,0.243985,0.266585,0.250466,-0.409792,NaN,NaN,NaN,NaN,NaN,NaN
4,0.243238,0.241783,0.259722,0.256650,-0.537654,0.248957,0.262436,0.276237,NaN,NaN,NaN
5,0.252315,0.237441,0.259880,0.243164,-0.481504,0.244431,0.256430,0.268690,NaN,NaN,NaN
6,0.257552,0.242272,0.268947,0.252241,-0.533332,0.225471,0.251277,0.277992,NaN,NaN,NaN
7,0.249067,0.242613,0.267053,0.257478,-0.541898,0.227273,0.251961,0.277506,NaN,NaN,NaN
8,0.251117,0.238782,0.267053,0.248993,-0.460833,0.227220,0.251667,0.276959,NaN,NaN,NaN
9,0.259758,0.244566,0.266347,0.251043,-0.424803,0.224929,0.250545,0.277063,NaN,NaN,NaN
